# 初期化・実行準備

In [1]:
!python3 -m venv .venv
!source .venv/bin/activate
!pip3 install -e ../

Obtaining file:///Users/huku/Desktop/fastlabel-python-sdk
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for fastlabel (pyproject.toml) ... done
  Created wheel for fastlabel: filename=fastlabel-0.15.9.dev11+ge041b8f-0.editable-py3-none-any.whl size=16887 sha256=a62f51e0752711335351f28434b0a1c219d317333a970a53a1e029ca49a90e94
  Stored in directory: /private/var/folders/lq/q0m2__ws4wx235q2zv7_hgv40000gn/T/pip-ephem-wheel-cache-jzco0p_8/wheels/b4/ca/40/3eabb73e3f7e9d1773d17d81a3333e468d3df67d969d4ddb08
Successfully built fastlabel
  Attempting uninstall: fastlabel
    Found existing installation: fastlabel 0.15.9.dev11+ge041b8f
    Uninstalling fastlabel-0.15.9.dev11+ge041b8f:
      Successfully uninstalled fastlabel-0.15.9.dev11+ge041b8f

[notice] A new release of pi

ipynbの実行環境を上で作成した仮想環境に変更する

# 簡易テスト実行

In [34]:
import fastlabel
import os
import json
from fastlabel.exceptions import FastLabelInvalidException

os.environ['FASTLABEL_ACCESS_TOKEN'] = "ZmE0MDk0OTctZWZmZS00MzZhLTg2ZTAtMmZkYjk5YjEwMmQ1"

SLUG = "fastlabel-sample"

client = fastlabel.Client()

if client.find_project_by_slug(SLUG):
    client.delete_project(client.find_project_by_slug(SLUG)["id"])

# Project    
project_id = client.create_project(type="image_bbox", name=SLUG, slug=SLUG)
if project_id:
    print("OK: Project created successfully")
else:
    raise Exception("Error: Project not created")
project = client.find_project_by_slug(SLUG)
if project:
    print("OK: Project found successfully")
else:
    raise Exception("Error: Project not found")
client.delete_project(project["id"])
if not client.find_project_by_slug(SLUG):
    print("OK: Project deleted successfully")
else:
    raise Exception("Error: Project not deleted")
project_id = client.create_project(type="image_bbox", name=SLUG, slug=SLUG)

# Annotation
annotation_id = client.create_annotation(
    project=SLUG, 
    type="bbox", 
    value="animal", 
    title="animal"
)
if annotation_id:
    print("OK: Annotation created successfully")
else:
    raise Exception("Error: Annotation not created")
annotation = client.find_annotation(annotation_id=annotation_id)
if annotation:
    print("OK: Annotation found successfully")
else:
    raise Exception("Error: Annotation not found")

# Task
with open("./sample_data/annotation/fastlabel/annotations.json", 'r') as file:
    data = json.load(file)
task_id = client.create_image_task(
    project=SLUG, 
    name="sample",
    file_path="./sample_data/images/sample1.jpg",
    annotations=data[0]["annotations"]
)
if task_id:
    print("OK: Task created successfully")
else:
    raise Exception("Error: Task not created")
task = client.find_image_task(task_id=task_id)
if task:
    print("OK: Task found successfully")
else:
    raise Exception("Error: Task not found")
client.delete_task(task_id)
try:
    task = client.find_image_task(task_id=task_id)
except FastLabelInvalidException as e:
    if "Task not found" in str(e):
        print("OK: Task deleted successfully")
    else:
        raise Exception("Error: Task not deleted")
except Exception as e:
    raise Exception("Error: Task not deleted")

# Dataset
dataset_id = next((dataset["id"] for dataset in client.get_datasets() if dataset["name"] == SLUG), None)
if dataset_id:
    client.delete_dataset(dataset_id=dataset_id)

dataset = client.create_dataset(
    name=SLUG,
)
if dataset:
    print("OK: Dataset created successfully")
else:
    raise Exception("Error: Dataset not created")

def remove_unnecessary_keys(annotations):
    keys_to_remove = ["id", "color"]
    clean_annotation = [
        {k: v for k, v in annotation.items() if k not in keys_to_remove}
        for annotation in annotations
    ]
    return clean_annotation

dataset_object = client.create_dataset_object(
    dataset=SLUG,
    name="sample.jpg",
    file_path="./sample_data/images/sample1.jpg",
    tags=["dog"],
    licenses=["MIT", "my-license"],
    annotations={
        "confidenceScore": 0,
        "rotation": 0,
        "points": [
            0
        ],
        "value": "animal",
        "type": "bbox"
    }
)
if dataset_object:
    print("OK: Dataset object created successfully")
else:
    raise Exception("Error: Dataset object not created")

dataset_id = next((dataset["id"] for dataset in client.get_datasets() if dataset["name"] == SLUG), None)
dataset_object = client.find_dataset_object(
    dataset_id=dataset_id,
    object_name="sample.jpg"
)
if dataset_object:
    print("OK: Dataset object found successfully")
else:
    raise Exception("Error: Dataset object not found")

client.delete_dataset(dataset_id=dataset_id)
try:
    dataset = client.find_dataset(dataset_id=dataset_id)
except FastLabelInvalidException as e:
    if "Not found error" in str(e):
        print("OK: Dataset deleted successfully")
    else:
        raise Exception("Error: Dataset not deleted")
except Exception as e:
    raise Exception("Error: Dataset not deleted")

OK: Project created successfully
OK: Project found successfully
OK: Project deleted successfully
OK: Annotation created successfully
OK: Annotation found successfully
OK: Task created successfully
OK: Task found successfully
OK: Task deleted successfully
OK: Dataset created successfully
OK: Dataset object created successfully
OK: Dataset object found successfully
OK: Dataset deleted successfully
